In [5]:
import logging
import warnings
warnings.filterwarnings("ignore") # ignore warnings
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

from transformers import TrOCRProcessor, VisionEncoderDecoderModel, pipeline
from PIL import Image


In [6]:
# question phrases
q_phrase = ["DL no.", "expiry date", "name", "address", "date of birth", "sex"]


In [7]:
def image_to_text(cropped_image):
    processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
    model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')
    pixel_values = processor(images=cropped_image, return_tensors="pt").pixel_values

    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids,
                                            skip_special_tokens=True)[0]
    return generated_text


def text_to_answer(text, bbox_label=None):
    model_name = "deepset/roberta-base-squad2"
    nlp = pipeline('question-answering',
                   model=model_name,
                   tokenizer=model_name)

    if bbox_label is None:
        bbox_label = q_phrase[0]
    elif bbox_label == "dl":
        bbox_label = q_phrase[0]
    elif bbox_label == "expiry":
        bbox_label = q_phrase[1]
    elif bbox_label == "name":
        bbox_label = q_phrase[2]
    elif bbox_label == "address":
        bbox_label = q_phrase[3]
    elif bbox_label == "dob":
        bbox_label = q_phrase[4]
    elif bbox_label == "gender":
        bbox_label = q_phrase[5]

    QA_input = {
        'question': "What is {}?".format(bbox_label),
        'context': '{}'.format(text.lower())
    }
    return nlp(QA_input)


def image_to_answer(cropped_image, bbox_label=None):
    text = image_to_text(cropped_image)
    answer = text_to_answer(text, bbox_label)
    return answer

In [8]:
# Driver code
crop_img = Image.open(r"./screenshot_5.png").convert("RGB")

# use bbox_label to specify the question phrase [dl, expiry, name, address, dob, gender] according to the cropped image
result = image_to_answer(crop_img, bbox_label="dob")['answer']
print(result)

The size parameter should be a dictionary with keys ('height', 'width'), ('shortest_edge', 'longest_edge') or ('shortest_edge',) got 384. Setting as {'height': 384, 'width': 384}.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


08/31/1977
